### Importing The Required Libraries:

In [1]:
import tensorflow as tf
import warnings
warnings.filterwarnings('ignore')
import pickle
import numpy as np

from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam


tf.__version__
sess = tf.compat.v1.Session(config=tf.compat.v1.ConfigProto(log_device_placement=True))
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

Device mapping:
/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: NVIDIA GeForce RTX 2080 SUPER, pci bus id: 0000:05:00.0, compute capability: 7.5



In [2]:
filename = "qa"
tokenizer = pickle.load(open('tokenizer/tokenizer-' + filename + '.pkl', 'rb'))


In [3]:
file = open("data/" + filename + ".txt", "r", encoding = "utf8")
lines = []
l = 0
for i in file:
    lines.append(i)
    l = l + 1
    if (l > 5):
        print("The First Line: ", lines[0])
        print("The Last Line: ", lines[-1])
    else:
        print(len(lines[0]))
data = ""

for i in lines:
    data = ' '. join(lines)

data = data.replace('\n', '').replace('\r', '').replace('\ufeff', '')

data[:360]

19687206


"Are you a fan of Google or Microsoft? Both are excellent technology they are helpful in many ways. For the security purpose both are super.  I'm not  a huge fan of Google, but I use it a lot because I have to. I think they are a monopoly in some sense.   Google provides online related services and products, which includes online ads, search engine and cloud "

In [4]:

sequence_data = tokenizer.texts_to_sequences([data])[0]
sequence_data[:10]

vocab_size = len(tokenizer.word_index) + 1
print(vocab_size)

36769


In [5]:

sequences = []

for i in range(1, int(len(sequence_data)/200)):
    words = sequence_data[i-1:i+1]
    sequences.append(words)
    
print("The Length of sequences are: ", len(sequences))
sequences = np.array(sequences)
sequences[:10]

The Length of sequences are:  18077


array([[  80,    3],
       [   3, 2607],
       [2607, 2116],
       [2116,  577],
       [ 577,  647],
       [ 647,  714],
       [ 714, 7019],
       [7019,  940],
       [ 940,   80],
       [  80, 6385]])

In [6]:
X = []
y = []

for i in sequences:
    X.append(i[0])
    y.append(i[1])
    
X = np.array(X)
y = np.array(y)
print("The Data is: ", X[:5])
print("The responses are: ", y[:5])

y = to_categorical(y, num_classes=vocab_size)
y[:5]

The Data is:  [  80    3 2607 2116  577]
The responses are:  [   3 2607 2116  577  647]


array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [7]:
model = Sequential()
model.add(Embedding(vocab_size, 10, input_length=1))
model.add(LSTM(1000, return_sequences=True))
model.add(LSTM(1000))
model.add(Dense(1000, activation="relu"))
model.add(Dense(vocab_size, activation="softmax"))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 1, 10)             367690    
_________________________________________________________________
lstm (LSTM)                  (None, 1, 1000)           4044000   
_________________________________________________________________
lstm_1 (LSTM)                (None, 1000)              8004000   
_________________________________________________________________
dense (Dense)                (None, 1000)              1001000   
_________________________________________________________________
dense_1 (Dense)              (None, 36769)             36805769  
Total params: 50,222,459
Trainable params: 50,222,459
Non-trainable params: 0
_________________________________________________________________


In [8]:

from tensorflow import keras
from tensorflow.keras.utils import plot_model


keras.utils.plot_model(model, to_file='model.png', show_layer_names=False)

('You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) ', 'for plot_model/model_to_dot to work.')


In [9]:
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.callbacks import TensorBoard

checkpoint = ModelCheckpoint("nextword-"+filename+".h5", monitor='loss', verbose=1,
    save_best_only=True, mode='auto')

reduce = ReduceLROnPlateau(monitor='loss', factor=0.2, patience=3, min_lr=0.0001, verbose = 1)

logdir='logsnextword'+filename
tensorboard_Visualization = TensorBoard(log_dir=logdir)

model.compile(loss="categorical_crossentropy", optimizer=Adam(learning_rate=0.001))

In [10]:
model.fit(X, y, epochs=50, batch_size=64, callbacks=[checkpoint, reduce, tensorboard_Visualization])

Epoch 1/50
283/283 [==============================] - 9s 18ms/step - loss: 7.1351

Epoch 00001: loss improved from inf to 7.13511, saving model to nextword-qa.h5
Epoch 2/50
283/283 [==============================] - 4s 16ms/step - loss: 6.2632

Epoch 00002: loss improved from 7.13511 to 6.26320, saving model to nextword-qa.h5
Epoch 3/50
283/283 [==============================] - 5s 16ms/step - loss: 6.1772

Epoch 00003: loss improved from 6.26320 to 6.17721, saving model to nextword-qa.h5
Epoch 4/50
283/283 [==============================] - 5s 16ms/step - loss: 6.0990

Epoch 00004: loss improved from 6.17721 to 6.09901, saving model to nextword-qa.h5
Epoch 5/50
283/283 [==============================] - 4s 16ms/step - loss: 5.9329

Epoch 00005: loss improved from 6.09901 to 5.93290, saving model to nextword-qa.h5
Epoch 6/50
283/283 [==============================] - 5s 17ms/step - loss: 5.7306

Epoch 00006: loss improved from 5.93290 to 5.73062, saving model to nextword-qa.h5
Epoch 7/

283/283 [==============================] - 4s 16ms/step - loss: 3.1461

Epoch 00050: loss improved from 3.15920 to 3.14614, saving model to nextword-qa.h5


In [12]:
# https://stackoverflow.com/questions/26649716/how-to-show-pil-image-in-ipython-notebook
# tensorboard --logdir="./logsnextword1"
# http://DESKTOP-U3TSCVT:6006/

from IPython.display import Image 
pil_img = Image(filename='graph1.png')
display(pil_img)